In [ ]:
# --- CELDA 1: CARGA Y PREPROCESAMIENTO (DATOS HISTÓRICOS 2015-2023) ---

import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# ==============================================================================
# 1. CARGA DE DATOS (LAS 3 FUENTES)
# ==============================================================================
try:
    df_matricula = pd.read_csv('../data/matricula_senescyt_2015_2023.csv')
    df_ofertas = pd.read_csv('../data/encuentra_empleo_ofertas_2.csv')
    df_inec = pd.read_csv('../data/inec_enemdu_salarios.csv')
    print("✅ Las 3 bases de datos han sido cargadas correctamente.")
except FileNotFoundError:
    print("❌ Error: Faltan archivos. Asegúrate de subir: SENESCYT, Ofertas e INEC.")

# 2. Procesar Estudiantes (TODA LA SERIE 2015-2023)
# Paso A: Agrupar por Carrera y Año primero (para obtener el total real por año)
df_anual = df_matricula.groupby(['carrera', 'año'])['num_estudiantes'].sum().reset_index()

# Paso B: Calcular métricas históricas por carrera
# Usamos el 'mean' (promedio) para obtener el flujo anual estimado de estudiantes.
# Esto nos dice: "En promedio, ¿cuántos estudiantes hay por año en esta carrera?"
df_estudiantes_agg = df_anual.groupby('carrera')['num_estudiantes'].mean().reset_index()
df_estudiantes_agg.rename(columns={'num_estudiantes': 'estudiantes_promedio_anual'}, inplace=True)

# 3. Procesar Ofertas
df_ofertas['salario_promedio'] = (df_ofertas['salario_minimo'] + df_ofertas['salario_maximo']) / 2
df_ofertas_agg = df_ofertas.groupby('carrera_requerida').agg({
    'titulo_puesto': 'count',
    'salario_promedio': 'mean'
}).reset_index()
df_ofertas_agg.rename(columns={'carrera_requerida': 'carrera', 'titulo_puesto': 'num_ofertas'}, inplace=True)

# 4. Merge (Left Join para no perder carreras sin ofertas)
df_estudiantes_agg['key'] = df_estudiantes_agg['carrera'].str.lower().str.strip()
df_ofertas_agg['key'] = df_ofertas_agg['carrera'].str.lower().str.strip()

df_final = pd.merge(df_estudiantes_agg, df_ofertas_agg, on='key', how='left')

# 5. Limpieza y Relleno de Nulos
df_final = df_final[['carrera_x', 'estudiantes_promedio_anual', 'num_ofertas', 'salario_promedio']]
df_final.rename(columns={'carrera_x': 'carrera'}, inplace=True)

# Asumimos 0 ofertas/salario para carreras no encontradas en el portal de empleo
df_final['num_ofertas'] = df_final['num_ofertas'].fillna(0)
df_final['salario_promedio'] = df_final['salario_promedio'].fillna(0)

print(f"✅ Análisis Histórico Procesado (2015-2023)")
print(f"Total Carreras: {len(df_final)}")
display(df_final.head())

✅ Archivos cargados.
✅ Análisis Histórico Procesado (2015-2023)
Total Carreras: 21


,carrera,estudiantes_promedio_anual,num_ofertas,salario_promedio
0,Administración de Empresas,1973.333333,268.0,1199.380597
1,Agronomía,242.111111,0.0,0.000000
2,Arquitectura,747.666667,0.0,0.000000
3,Biología,486.333333,0.0,0.000000
4,Comunicación Social,476.000000,252.0,992.091270


In [ ]:
# --- CELDA 2: DÍA 5 - CLUSTERING (ACUMULADO HISTÓRICO 2015-2023) ---
# 1. PREPROCESAMIENTO: Total Histórico de Estudiantes
# Agrupamos por carrera y SUMAMOS todos los registros desde 2015
# Esto nos da la "masa total" de profesionales formados
df_historico = df_matricula.groupby('carrera')['num_estudiantes'].sum().reset_index()

# 2. Procesamiento de Ofertas (Snapshot Actual)
df_ofertas['salario_promedio'] = (df_ofertas['salario_minimo'] + df_ofertas['salario_maximo']) / 2
df_ofertas_agg = df_ofertas.groupby('carrera_requerida').agg({
    'titulo_puesto': 'count',
    'salario_promedio': 'mean'
}).reset_index()
# Normalización de nombres para el cruce
df_ofertas_agg['key'] = df_ofertas_agg['carrera_requerida'].str.lower().str.strip()
df_historico['key'] = df_historico['carrera'].str.lower().str.strip()

# 3. MERGE (Unión usando el Total Acumulado)
df_final = pd.merge(df_historico, df_ofertas_agg, on='key', how='left')

# Limpieza y manejo de nulos (Carreras sin ofertas = 0)
df_final = df_final[['carrera', 'num_estudiantes', 'titulo_puesto', 'salario_promedio']]
df_final.rename(columns={'titulo_puesto': 'num_ofertas'}, inplace=True)
df_final['num_ofertas'] = df_final['num_ofertas'].fillna(0)
df_final['salario_promedio'] = df_final['salario_promedio'].fillna(0)

# 4. CLUSTERING (K-MEANS)
# Features oficiales del documento
features = ['num_estudiantes', 'num_ofertas', 'salario_promedio']
X = df_final[features]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
df_final['cluster'] = kmeans.fit_predict(X_scaled)

# 5. ETIQUETADO DE CLUSTERS
# Analizamos centroides vs Media Global
centroides = df_final.groupby('cluster')[features].mean()
medias = df_final[features].mean()

def definir_etiqueta(row, media_global):
    # Comparación estricta
    es_demandada = row['num_ofertas'] > media_global['num_ofertas']
    es_bien_pagada = row['salario_promedio'] > media_global['salario_promedio']
    es_populosa = row['num_estudiantes'] > media_global['num_estudiantes']

    if es_demandada and es_bien_pagada:
        return "En Demanda"
    elif es_populosa and not es_demandada:
        return "Saturadas"  # Muchos estudiantes acumulados, pocas ofertas
    elif not es_populosa and (es_bien_pagada or es_demandada):
        return "Nicho"
    else:
        return "Balanceadas"

# Aplicamos etiquetas
mapa = {i: definir_etiqueta(centroides.loc[i], medias) for i in centroides.index}
df_final['categoria'] = df_final['cluster'].map(mapa)

# 6. VISUALIZACIÓN FINAL
print("Generando gráfico con volumen histórico total...")
fig = px.scatter_3d(
    df_final,
    x='num_estudiantes',
    y='num_ofertas',
    z='salario_promedio',
    color='categoria',
    symbol='categoria',
    hover_name='carrera',
    title='<b>Clustering de Carreras (Acumulado 2015-2023)</b>',
    labels={
        'num_estudiantes': 'Total Estudiantes (2015-2023)',
        'num_ofertas': 'Ofertas Actuales',
        'salario_promedio': 'Salario promedio ($)'
    },
    color_discrete_map={
        "En Demanda": "#00CC96",
        "Saturadas": "#EF553B",
        "Nicho": "#636EFA",
        "Balanceadas": "#FECB52"
    },
    opacity=0.9
)
fig.update_layout(
    scene=dict(
        # Eje X (Estudiantes)
        xaxis=dict(
            title=dict(font=dict(size=10)),  # Tamaño del título (ej: "Total Estudiantes")
            tickfont=dict(size=9)            # Tamaño de los números
        ),
        # Eje Y (Ofertas)
        yaxis=dict(
            title=dict(font=dict(size=10)),
            tickfont=dict(size=9)
        ),
        # Eje Z (Salarios)
        zaxis=dict(
            title=dict(font=dict(size=10)),
            tickfont=dict(size=9)
        ),
    ),
    # Ajustar márgenes para que no se corte si el texto es pequeño
    margin=dict(l=0, r=0, b=0, t=30),
    legend=dict(font=dict(size=10))
)

fig.show()

Generando gráfico con volumen histórico total...
